# LLM finetuning experiment with Llama 3
In this experiment, we build upon results from a previous experiment conducted by [Snorkel](https://snorkel.ai/few-shot-learning-large-language-models/). Snorkel shows us how using few-shot learning, where adding a few "shots" of labeled examples to an LLM (in this case, BERT) and then incorporating it into Snorkel with human labeling functions can be more effective than either on their own. Here, we expand upon these results by testing few-shot learning with a vastly more powerful open source model, Meta's Llama 3.

### Creating a labeled subset of data for fine-tuning
We use [Unsloth](https://github.com/unslothai/unsloth) to fine-tune Llama 3. Unsloth promises 2x faster performance with 60% less memory for training the 8B version of Llama 3. In order to train it, we need to build a JSON file with three fields for each object: instruction, input, and output. I manually took the last 15 items in the training dataset and converted them into this format. After reviewing them, I found one article which had a classification decision that I disagreed with. It classified a "deep learning" reserch article as non-CS, choosing Statistics and Quantitative Finance instead. This classification is debatable, but I worried it might confuse the LLM to not classify AI/ML articles as CS, so I removed it and included the 16th article from the end instead.

There are 9 non-CS articles and 6 CS articles. This proportion seems good to me because the model has a tendency to classify non-CS articles as CS, so explicitly showing how many articles are indeed non-CS might improve its accuracy.

After fine-tuning Llama 3, we get two models in the GGUF file format: 1) a quantized version (4 GB), and 2) a larger F16 version (16 GB). The larger one seems quite slow, so we use the quantized version for now.

### Generate labels

In [3]:
import pandas as pd
import ollama

In [4]:
df = pd.read_csv("data/text-classification-train.csv")
df.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [5]:
df_labeled = pd.DataFrame()
df_labeled["abstract"] = df["ABSTRACT"][:200]
df_labeled["CS_actual"] = df["Computer Science"][:200]
df_labeled

,abstract,CS_actual
0,Predictive models allow subject-specific inf...,1
1,Rotation invariance and translation invarian...,1
2,We introduce and develop the notion of spher...,0
3,The stochastic Landau--Lifshitz--Gilbert (LL...,0
4,Fourier-transform infra-red (FTIR) spectra o...,1
...,...,...
195,We relate the concepts used in decentralized...,0
196,Time-varying network topologies can deeply i...,1
197,A long-standing obstacle to progress in deep...,1
198,We study the band structure topology and eng...,0


In [1]:
def label_abstract(x):
    prompt = f'''I will paste an abstract from a research article. I want you
        to respond "1" if the abstract comes from a paper in the computer science field, and "0"
        if it doesn't or if you are not sure. Do not include anything else in your response; just
        the number. Please classify this abstract: {x}'''
    response = ollama.chat(model='llama3_ft2', messages=[
      {
        'role': 'user',
        'content': prompt,
      },
    ])
    return response['message']['content']

In [12]:
label_abstract(df_labeled["abstract"].loc[0])

KeyboardInterrupt: 

In [9]:
label_abstract(df_labeled["abstract"].loc[2])

'    Your response:\n'

In [10]:
label_abstract(df_labeled["abstract"].loc[3])

''